<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Forex_Trader_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import datetime
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler

# Step 2: Gathering Data for Your Forex Trading Bot
#symbol = "EURUSD=X"  # Example currency pair (EUR/USD)
#symbol = "USDJPY=X"  # Example currency pair (USD/JPY)
#symbol = "GBPUSD=X"
#symbol = "NZDUSD=X"
#symbol = "AUDUSD=X"
symbol = "AUDNZD=X"

start_date = "2010-01-01"
# Get the current date
end_date = datetime.datetime.now().strftime("%Y-%m-%d")
#end_date = "2023-07-19"
df = yf.download(symbol, start=start_date, end=end_date, interval="1d")  # Daily interval

# Preprocess the data
# Add necessary preprocessing steps for forex data if required

# Split the data into training and testing sets
train_size = int(len(df) * 0.8)
train_data = df[:train_size]
test_data = df[train_size:]


# Define a function to create input sequences and labels
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequence = data[i : i + seq_length]
        label = data[i + seq_length]
        sequences.append(sequence)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# Convert train_data and test_data to NumPy arrays
train_data = train_data["Close"].values
test_data = test_data["Close"].values

# Create input sequences and labels
seq_length = 60
train_x, train_y = create_sequences(train_data, seq_length)
test_x, test_y = create_sequences(test_data, seq_length)

# Continue with the rest of the code for building, training, and using the Forex Trading Bot

# Continue with the rest of the code for building, training, and using the Forex Trading Bot

# Step 3: Building Your Stock Trading Bot
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(train_x.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

# Train the model
model.fit(train_x, train_y, epochs=10, batch_size=32)

# Step 4: Evaluating Your Stock Trading Bot
# Evaluate the model on the test set
test_loss = model.evaluate(test_x, test_y)

# Step 5: Using Your Stock Trading Bot
# Make predictions on the test set
# Create a new scaler for inverse transforming the predictions
scaler = MinMaxScaler()
scaler.fit(df["Close"].values.reshape(-1, 1))

predictions = model.predict(test_x)
predictions = scaler.inverse_transform(predictions)

# Convert the predictions and actual values to pandas DataFrame
predictions_df = pd.DataFrame(predictions, index=df.index[train_size+seq_length:])
actual_df = pd.DataFrame(scaler.inverse_transform(test_y.reshape(-1, 1)), index=df.index[train_size+seq_length:])

# Compare the predictions and actual values
comparison_df = pd.concat([actual_df, predictions_df], axis=1)
comparison_df.columns = ["Actual", "Predicted"]
print(comparison_df)

# Save the model for deployment
model.save("stock_trading_bot_model.keras")

# Create input sequences and labels
seq_length = 60
train_x, train_y = create_sequences(train_data, seq_length)
test_x, test_y = create_sequences(test_data, seq_length)

# Step 3: Building Your Stock Trading Bot
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(train_x.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Define the input shape for predicting the next day
next_day_sequence = np.expand_dims(test_x[-1], axis=0)

# Make a prediction for the next day
next_day_prediction = model.predict(next_day_sequence)
next_day_price = scaler.inverse_transform(next_day_prediction)[0][0]

# Print the predicted price for the next day
print("Predicted price for the next day:", next_day_price)

# Determine the action based on the predicted price
if next_day_price > df["Close"].iloc[-1]:
    print("Action: Buy")
elif next_day_price < df["Close"].iloc[-1]:
    print("Action: Sell")
else:
    print("Action: Hold")


[*********************100%***********************]  1 of 1 completed
Epoch 1/10
87/87 [==============================] - 8s 50ms/step - loss: 0.0424
Epoch 2/10
87/87 [==============================] - 5s 54ms/step - loss: 7.6847e-04
Epoch 3/10
87/87 [==============================] - 4s 51ms/step - loss: 4.4182e-04
Epoch 4/10
87/87 [==============================] - 5s 52ms/step - loss: 2.2120e-04
Epoch 5/10
87/87 [==============================] - 4s 52ms/step - loss: 1.5478e-04
Epoch 6/10
87/87 [==============================] - 5s 53ms/step - loss: 1.4623e-04
Epoch 7/10
87/87 [==============================] - 5s 52ms/step - loss: 1.4517e-04
Epoch 8/10
87/87 [==============================] - 5s 54ms/step - loss: 1.3806e-04
Epoch 9/10
87/87 [==============================] - 4s 46ms/step - loss: 1.4634e-04
Epoch 10/10
21/21 [==============================] - 1s 13ms/step
              Actual  Predicted
Date                           
2021-01-26  1.401617   1.400359
2021-01-27  1.401